
Create a Minecraft Server on Google Colab!

The script below will initiate your server. First, you will need to create the server to be able to use it - don't worry, the scripts below will do the majority of the work for you. Perhaps you want to change the server's region, check below.

Original:https://github.com/agrawalchaitany/minecolab.git

Modified Version Well Optimized

```
███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗
████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝
██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝

```


# **1º Create the Server (Only do this the first time)**

The code below will create your server and accept the EULA. When you use these scripts, your server will be ready to start.


**Download the Minecraft Server**

The code below will download Paper, a type of high-performance Vanilla server that allows the use of plugins. If you want to use another type of server.jar, put it in the Drive folder manually.

The server files will be saved on your Google Drive (in the Google account you choose or are using).

In [ ]:
# Change "1.21.4" to the desired version.
# Available tested versions:
# - 1.21.4
# Newer versions might work too, however this isn't guaranteed.

version = '1.21.4'

# Choose server type. Currently available versions:
# - paper (Most vanilla. It's the same as SPIGOT, bukkit, etc.)
# - forge (For using MODS. It's basically FORGE) [Note, installation takes approximately 10-15 minutes, wait for "COMPLETED" after executing the first step.]
# - fabric (For playing Fabric. It's an alternative to Forge)
# - purpur (A highly customizable and enhanced Paper server)

server_type = 'paper'

from google.colab import drive
import requests
import json

drive.mount('/content/drive')

# Create the directory which will be used for the server
!mkdir -p "/content/drive/My Drive/Minecraft-server"
%cd "/content/drive/My Drive/Minecraft-server"

# Internal init...

paperURL = None
if server_type == 'paper':
    paperURL = "https://piston-data.mojang.com/v1/objects/4707d00eb834b446575d89a61a11b5d548d8c001/server.jar"

purpurURL = None
if server_type == 'purpur':
    c = requests.get("https://api.purpurmc.org/v2/purpur/" + version)
    response_json = c.json()
    if 'builds' in response_json and 'all' in response_json['builds'] and len(response_json['builds']['all']) > 0:
        try:
            latest_build = response_json["builds"]["latest"]
            d = requests.get("https://api.purpurmc.org/v2/purpur/" + version + "/" + str(latest_build))
            purpurURL = "https://api.purpurmc.org/v2/purpur/" + version + "/" + str(latest_build) + "/download"
        except KeyError as e:
            print(f"KeyError occurred: {e}")
            print(f"Response JSON: {response_json}")
            exit()
    else:
        print(f"No builds found for Purpur version {version}. Please check the version number and try again.")
        exit()

forgeURL = "https://serverjars.com/api/fetchJar/modded/forge/" + version

jar_name = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar', 'forge': 'forge-installer.jar', 'purpur': 'purpur.jar'}
url = {
    'paper': paperURL,
    'fabric': 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.7.4/fabric-installer-0.7.4.jar',
    'forge': forgeURL,
    'purpur': purpurURL
}

print('Downloading to Google Drive...')

if url[server_type] is None:
    print(f"Invalid URL for {server_type} server. Please check the configuration.")
else:
    r = requests.get(url[server_type])
    if r.status_code == 200:
        with open('/content/drive/My Drive/Minecraft-server/' + jar_name[server_type], 'wb') as f:
            f.write(r.content)
    else:
        print(f'Error {r.status_code}! Most likely you entered an unsupported version. Try running the code again if you think that shouldn\'t have happened.')

# Running specific install path.
if server_type == 'fabric':
    !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

if server_type == 'forge':
    %cd "/content/drive/My Drive/Minecraft-server"
    !java -jar forge-installer.jar --installServer

# Saving config
colabconfig = {"server_type": server_type,
               "server_version": version}
json.dump(colabconfig, open("colabconfig.json", 'w'))

print('Completed!')  # todo: Would show even after erroring.

# Please read the file stored in your server folder before running this command.
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server
Completed!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server


# **-> Start the Minecraft server - Use this whenever you want to open it.**



In [ ]:
import os
import time
import json
import re
from google.colab import drive

# 🛠️ Update package lists
print("\n🔄 Updating package lists...")
os.system("sudo apt update &>/dev/null && echo '✅ apt cache successfully updated' || echo '❌ apt cache update failed.'")

# 📂 Mount Google Drive
print("\n🔗 Mounting Google Drive...")
drive.mount('/content/drive')

# 🎮 Change directory to Minecraft server folder
server_path = "/content/drive/My Drive/Minecraft-server"
if os.path.exists(server_path):
    os.chdir(server_path)
    print(f"✅ Changed working directory to: {server_path}")
    print("📂 Listing server files:\n")
    os.system("ls")
else:
    print(f"❌ Error: {server_path} not found. Make sure the folder exists in Google Drive.")
    exit()

# 🔧 Load or create configuration file
config_file = "colabconfig.json"
if os.path.isfile(config_file):
    with open(config_file, "r") as f:
        colabconfig = json.load(f)
else:
    colabconfig = {"server_type": "generic"}
    with open(config_file, "w") as f:
        json.dump(colabconfig, f)

server_type = colabconfig.get("server_type", "generic")

# ☕ Install OpenJDK 21
print("\n🔧 Installing OpenJDK 21...")
os.system("sudo apt-get install openjdk-21-jre-headless -y &>/dev/null")

# ✅ Verify Java installation
java_version = os.popen("java -version 2>&1 | awk -F '\"' 'NR==1 {print $2}'").read().strip()
if java_version.startswith("21"):
    print(f"✅ OpenJDK {java_version} is working correctly.")
else:
    print(f"⚠️ OpenJDK 21 is not installed or not working. Detected Java version: {java_version}")
    exit()

# 🗂️ Server jar names
jar_list = {
    'paper': 'server.jar',
    'fabric': 'fabric-server-launch.jar',
    'generic': 'server.jar',
    'forge': 'forge.jar',
    'purpur': 'purpur.jar'
}
jar_name = jar_list.get(server_type, "server.jar")

# 📥 Download Purpur server jar if needed
if server_type == "purpur" and not os.path.isfile("purpur.jar"):
    print("\n📥 Downloading Purpur server jar...")
    os.system("curl -o purpur.jar https://api.purpurmc.org/v2/purpur/1.21/latest/download")

# 🔧 Java arguments for better performance
if server_type in ["paper", "purpur"]:
    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
    server_flags = ""

memory_allocation = "-Xms8704M -Xmx8704M"

# 🌍 Setup tunneling service
tunnel_service = "nglocalhost"  # Can be changed to "playit.gg" if needed
print(f"\n🌍 Using {tunnel_service} for server tunneling...")

# 🔌 Establishing the tunnel
if tunnel_service == "nglocalhost":
    print("\n🔌 Setting up nglocalhost tunneling...")
        # Generate SSH key if missing
    if not os.path.isfile("/content/drive/My Drive/Minecraft-server/ssh_keys/id_rsa"):
        os.system("mkdir -p '/content/drive/My Drive/Minecraft-server/ssh_keys'")  # Create directory
        os.system("ssh-keygen -t rsa -b 4096 -f '/content/drive/My Drive/Minecraft-server/ssh_keys/id_rsa' -N '' -q")  # Generate SSH key
        os.system("chmod 400 '/content/drive/My Drive/Minecraft-server/ssh_keys/id_rsa'")  # Change permissions
        os.system("ssh-keygen -lf '/content/drive/My Drive/Minecraft-server/ssh_keys/id_rsa.pub' > '/content/drive/My Drive/Minecraft-server/ssh_keys/id_rsa_fingerprint.txt'" )

    # Establish tunnel
    print("⏳ Establishing tunnel, please wait...")
    os.system("nohup ssh -T -o StrictHostKeyChecking=no -i '/content/drive/My Drive/Minecraft-server/ssh_keys/id_rsa' -R minecraft03127:25565:localhost:25565 nglocalhost.com > tunnel_log.txt 2>&1 &")

    # Wait for tunnel to establish
    time.sleep(10)

    # Extract and display the tunnel URL
    with open("tunnel_log.txt", "r") as log_file:
        log_content = log_file.read()
        match = re.search(r"(nglocalhost[^\s]+)", log_content)  # Extract first URL
        if match:
            tunnel_url = match.group(0)
            print(f"✅ Tunnel established! Access your Minecraft server at:\n🌍 {tunnel_url}")
        else:
            print("⚠️ Could not extract tunnel URL. Check 'tunnel_log.txt' manually.")

else:
    print("⚠️ No tunneling service selected. Server will run locally.")

# 🚀 Start Minecraft server
print("\n🚀 Starting Minecraft server...")
!java {memory_allocation} {server_flags} -jar {jar_name} nogui
print("✅ Minecraft server closed!")


🔄 Updating package lists...

🔗 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Changed working directory to: /content/drive/My Drive/Minecraft-server
📂 Listing server files:


🔧 Installing OpenJDK 21...
✅ OpenJDK 21.0.5 is working correctly.

🌍 Using nglocalhost for server tunneling...

🔌 Setting up nglocalhost tunneling...
⏳ Establishing tunnel, please wait...
✅ Tunnel established! Access your Minecraft server at:
🌍 nglocalhost.com:25565

🚀 Starting Minecraft server...
Starting net.minecraft.server.Main
[04:41:13] [ServerMain/INFO]: Environment: Environment[sessionHost=https://sessionserver.mojang.com, servicesHost=https://api.minecraftservices.com, name=PROD]
[04:41:17] [ServerMain/INFO]: Loaded 1370 recipes
[04:41:17] [ServerMain/INFO]: Loaded 1481 advancements
[04:41:17] [Server thread/INFO]: Starting minecraft server version 1.21.4
[04:41:17] [Server thread/INFO]: Loading p

**FAQ - THINGS YOU NEED TO KNOW**

you just need to download playit.gg plugin and from there you can make a static ip.

open google drive for adding plugins and other files modification.